In [1]:
%matplotlib inline
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Mm
# import tkinter as tk
# from tkinter import *
# #from tkinter import simpledialog
# from tkinter import messagebox
# from tkinter import filedialog
# from tkinter import ttk
#import ntpath
import datetime
import docx
import pandas as pd
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interactive
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
from IPython.display import display, clear_output
import voila
from ipyfilechooser import FileChooser

class Report_template:
    device_des = ''
    freq = ''
    sup_vol = ''
    output_type = ''
    dev_code = ''
    rev_day = ''
    rev_month = ''
    rev_year = ''
    n = ''
    so_num = ''
    name = ''
    test_cond = ''
    test1_cond = ''
    test2_cond = ''
    test_set_1 = ''
    test1_set_1 = ''
    test2_set_1 = ''
    fvt_spec = 0
    temp_range = [0]
    plot_num = 0
    pn_spec = [0]
    save_path = ''
    ebt_num = 0
    pn_fl_type = 'csv'
    
    
temp = Report_template()
template = 0
cols_2 = []
fvt_limX = 0
save_button = 0
ok_button = 0
out = 0
fvt_limX = 0
fvt_limY_pos = 0
fvt_limY_neg = 0
context = 0
context_con = 0

def Merge_context(contxt1, contxt2):
    contxt_full = {**contxt1, **contxt2}
    return contxt_full


def get_Table_1(template_path):
    table_path = ebt_data.selected
    # ebt_num 
    #rows_2_read =[*range(6,10+int(temp.ebt_num),1)]
    rows_2_read =[*range(10,10+int(temp.ebt_num),1)]
    rows_2_read_temp =[*range(6,9,1)]
    read_file = pd.read_csv (table_path, skiprows= lambda x:x not in rows_2_read, delim_whitespace=False)
    read_file_temp = pd.read_csv (table_path, skiprows= lambda x:x not in rows_2_read_temp, delim_whitespace = False)
    
    read_file.to_csv ('BenchMeasurements.csv', index=False)
    read_file_temp.to_csv ('BenchMeasurements_temp.csv', index=False)

    fl = pd.read_csv ('BenchMeasurements.csv', index_col = False)
    fl1 = pd.read_csv ('BenchMeasurements_temp.csv', index_col = False)
    #print(fl)
    fl = fl.to_string(index=False)
    fl1 = fl1.to_string(index=False)
    #fl=fl.replace("  ", "      ")
    #fl=fl.replace(" ", "      ")
    #fl=fl.replace("      "*6, "      ")
    #fl=fl.replace("            ", "      ")
    #fl=fl.replace("NaN", "")
    #print(fl)
    fl1=fl1.replace("                  ", "                            ")
    fl1=fl1.replace("  ", "    ")
    fl1=fl1.replace("             ", "       ")
    fl1=fl1.replace("                                ", "                                     ")
    fl1=fl1.replace("          ", "                ")
    fl1=fl1.replace("         ", "        ")
    fl1=fl1.replace("                                                 ", "                                      ")
    fl=fl.replace(" ", "  ")

    mydoc = docx.Document(template_path)

    # t=mydoc.add_table(fl.shape[0]+1,fl.shape[1])
    # for j in range(fl.shape[-1]):
        #     t.cell(0,j).text=fl.columns[j]
    
    # for i in range(fl.shape[0]):
        #     for j in range(fl.shape[-1]):
            #         t.cell(i+1,j).text = str(fl.values[i,j])

    # mydoc.save("C:/Xice/python_excel/template_test.docx")
    # doc = docx.Document() 

    #doc.save('C:/Xice/python_excel/table.docx')
    for para in mydoc.paragraphs:
        if 'TABLE_1' in para.text:
            para.text = fl1
            #print(para.text)
        if 'TABLE_2' in para.text:
            para.text = fl
      
    # mydoc.save("C:/Xice/python_excel/template_test.docx")
    mydoc.save(template_path)
    
    
def PN_gen_csv():
    # plot_num
    d = {}
    for num in range(temp.plot_num):
        file_path = pn_data.selected
        print(file_path)
        # unit_no = file_path[-12:-4:1] # To present Unit marked No#
        unit_no = "Unit" + str(num+1) # To present Unit No# in sequence
        print(unit_no)
        d[num]=pd.read_csv(file_path)
        d[num].rename(columns = {'Phase Noise (dBc/Hz)' : 'Phase Noise (dBc/Hz) - '+ unit_no}, inplace = True)
        print(d[num])
        #d[num].iloc[:,1] = d[num].iloc[:,1].rolling(15).mean() # Calculating Simple Moving Average
        d[num].iloc[:,1] = d[num].iloc[:,1].ewm(span = 30).mean() # Calculating Exponential Moving Average
        #x = list(d[num]['Offset Frequency (Hz)'])
        #y = list(d[num]['Phase Noise (dBc/Hz)'])
        lst = [1, 10, 100, 1000, 10000, 100000, 1000000] 
        #lst = [10, 100, 1000, 10000, 100000, 1000000] # as per SPEC
        df_spec = pd.DataFrame()
        df_spec['Offset'] = lst
        df_spec['PN typical'] = temp.pn_spec

# df = pd.concat([d[0], d[1]])
# print(df)
    
# # x = list(df.iloc[0:,0])
# # y = list(df.iloc[0:,1])
# print(x)
# print(y)
    for i in range(temp.plot_num):
        if i == 0:
            ax = df_spec.plot(x = 'Offset', y = 'PN typical', kind ='line', linestyle='dashed', logx = True, figsize = (16, 8), grid = True)
            d[i].plot(ax = ax, x = 'Offset Frequency (Hz)', y = d[i].columns[1], kind ='line', ylabel = 'Phase Noise(dBc/Hz)', title='Phase Noise', logx = True, figsize = (16, 8), grid = True)
        if(i > 0):
                d[i].plot(ax = ax, x = 'Offset Frequency (Hz)', y = d[i].columns[1], kind ='line', ylabel = 'Phase Noise(dBc/Hz)', title='Phase Noise', logx = True, figsize = (16, 8), grid = True)
    fig = ax.get_figure()
    fig.savefig("PN_output.png")
    
    
    
def PN_gen_txt():
    # plot_num
    d = {}
    for num in range(temp.plot_num):
        file_path = pn_data.selected
        print(file_path)
        #unit_no = file_path[-12:-4:1] # To present Unit marked No#
        unit_no = "Unit" + str(num+1) # To present Unit No# in sequence
        print(unit_no)
        read_file = pd.read_csv (file_path, delim_whitespace = True)
        read_file.to_csv(str(unit_no)+'.csv', index=None)
        file_path = str(unit_no)+'.csv'
        d[num]=pd.read_csv(file_path)
        d[num] = d[num][:-1]
        d[num].columns = ['Offset Frequency (Hz)', 'Phase Noise (dBc/Hz) - '+ unit_no]
        print(d[num])
        #d[num].iloc[:,1] = d[num].iloc[:,1].rolling(15).mean() # Calculating Simple Moving Average
        d[num].iloc[:,1] = d[num].iloc[:,1].ewm(span = 30).mean() # Calculating Exponential Moving Average
        #print(d[num])
        #x = list(d[num]['Offset Frequency (Hz)'])
        #y = list(d[num]['Phase Noise (dBc/Hz)'])
        lst = [1, 10, 100, 1000, 10000, 100000, 1000000]
        #lst = [10, 100, 1000, 10000, 100000, 1000000] # as per SPEC
        df_spec = pd.DataFrame()
        df_spec['Offset'] = lst
        df_spec['PN typical'] = temp.pn_spec

# df = pd.concat([d[0], d[1]])
# print(df)
    
# # x = list(df.iloc[0:,0])
# # y = list(df.iloc[0:,1])
# print(x)
# print(y)
    for i in range(temp.plot_num):
        if i == 0:
            ax = df_spec.plot(x = 'Offset', y = 'PN typical', kind ='line', linestyle='dashed', logx = True, figsize = (16, 8), grid = True)
            d[i].plot(ax = ax, x = 'Offset Frequency (Hz)', y = d[i].columns[1], kind ='line', ylabel = 'Phase Noise(dBc/Hz)', title='Phase Noise', logx = True, figsize = (16, 8), grid = True)
        if(i > 0):
                d[i].plot(ax = ax, x = 'Offset Frequency (Hz)', y = d[i].columns[1], kind ='line', ylabel = 'Phase Noise(dBc/Hz)', title='Phase Noise', logx = True, figsize = (16, 8), grid = True)
    fig = ax.get_figure()
    fig.savefig("PN_output.png")
    
    
    
def FvT_gen():
    file_path = fvt_data.selected
    print(file_path)
    if(file_path[-3:] == 'csv'):
        read_file = pd.read_csv(file_path)
        file_path = 'fvt_csv_cvted' + '.xlsx'
        print(file_path)
        read_file.to_excel(file_path,index = None)
        
    wb = load_workbook(filename= file_path, read_only=True)
    sheet = wb.sheetnames
    for i in sheet:
        print(i)
    ws = wb[sheet[0]]
    data_rows = []
    for row in ws.iter_rows(ws.min_row, ws.max_row):
    # for row in ws['A1':'E5727']:
        data_cols = []
        for cell in row:
            data_cols.append(cell.value)
        data_rows.append(data_cols)
    print(ws.max_row)
    df = pd.DataFrame(data_rows)
    #print(df)
    row_cnt = df.shape[0]
    
    #print(df.drop(df.columns[range(9)], axis = 1))
    df = df.drop(df.columns[range(9)], axis = 1)
    print(df.T)
    df = df.T # transpose dataframe from col to row
    cols = [1]
    FvT_gen.df2 = df[df.columns[cols]]
    #print(FvT_gen.df2)
    global cols_2
    for i in range(2,row_cnt,3): #get needed data rows/col
        cols_2.append(i)
    #print(cols_2)
    df = df[df.columns[cols_2]]
    #print(df)
    FvT_gen.result = pd.concat([FvT_gen.df2, df],axis=1)
    #print(FvT_gen.result)
    
    FvT_gen.result = FvT_gen.result.T.reset_index(drop=True).T ##reset columns index starts from 0
    FvT_gen.result = FvT_gen.result.astype('float64') ## rounding the dataframe decimal place
    FvT_gen.result.round(1)
    print(FvT_gen.result)
    
    ## res created for later manipulation
    FvT_gen.res = FvT_gen.df2.T.reset_index(drop=True).T
    FvT_gen.res = FvT_gen.res.astype('float64')
    FvT_gen.res.round(1)
    print(FvT_gen.res)
    

##--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def ok_on_click(b):
    with out:
        clear_output()
        #global ok_flag
        #ok_flag = 1
        # Rename column labels in unitNo# sequence
        global context, context_con
        unit_qty = len(FvT_gen.res.columns)-1
        col_lst = ["Operating Temperature(degC)"]
        for _ in range(unit_qty):
            col_lst.append("Unit"+ str(_+1))

        print(col_lst)
        FvT_gen.res.columns = col_lst
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

        ## finalised plot generated according to saved Units data
        plt =FvT_gen.res.plot(x = FvT_gen.res.columns[0], y = FvT_gen.res.columns[1:], kind = 'line', figsize = (20, 10), ylabel = 'Norm.Freq(ppm)', title='Sample Frequency Stability', grid = True)
        #plt =FvT_gen.res.plot(x = 0, y = FvT_gen.res.columns[1:], kind = 'line', figsize = (20, 10), ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
        plt.plot(fvt_limX, fvt_limY_pos, '--r', label = 'FvT Limit+'+ '(' + str(temp.fvt_spec) + 'ppm)')
        plt.plot(fvt_limX, fvt_limY_neg, '--r', label = 'FvT Limit-'+ '(' + str(-temp.fvt_spec) + 'ppm)')

        plt.set_xlim(xmin = temp.temp_range[0], xmax = temp.temp_range[1]) # show preferred x-axis range as per operating temperature
        plt.set_ylim(ymin = -temp.fvt_spec*2.5, ymax = temp.fvt_spec*2.5) # Keep the plot centered within scope

        #plt.xaxis.set_ticks(np.arange(-45, 105, 20))
        #plt.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
        plt.legend()
        fig = plt.get_figure()
        fig.savefig("FvT_output_0" +'.png')
##---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        global template
        context['FvT_Plot_0'] = InlineImage(template, 'FvT_output_0.png', width = Mm(240), height= Mm(120))
        #context['FvT_Plot_1'] = InlineImage(template, 'FvT_output_1.png', width = Mm(240), height= Mm(120))
##---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        if(temp.pn_fl_type == 'csv'):
            PN_gen_csv()
        if(temp.pn_fl_type == 'txt'):
            PN_gen_txt()
    
        context['PN_Plot'] = InlineImage(template, 'PN_output.png', width = Mm(240), height= Mm(120))

        template.render(Merge_context(context, context_con))
        # save_path = simpledialog.askstring(title="Save", prompt="Please type in the file name with .docx : \t\t\t\t\t\t")
        template.save(temp.save_path)
        get_Table_1(temp.save_path)
        
def save_on_click(b):
    with out:
        #FvT_gen.df2.plot(x = 'Temp(deg)', y = UnitNo, kind = 'line', figsize = (16, 5)).savefig(f'FvT.png')
        #print(FvT_gen.df2[b])
        #print(111)
        clear_output()
        #print(FvT_gen.result[temp_data])
        print("\n.............................................................................................\n")
        print("\nYOUR SAVED UNIT DATA:\n")
        FvT_gen.res[temp_data] = FvT_gen.result[temp_data] ## update selected Unit data into the df3
        print(FvT_gen.res)
        global saved_cnt
        saved_cnt = len(FvT_gen.res.columns)-1
        print("Saved Counter - ", saved_cnt)
        
def multiplot(UnitNo):
    global temp_data ## the global 'temp' variable 
    opts = FvT_gen.result.columns.values
    #plt = FvT_gen.result.plot(x = 0, y = 1, kind = 'line', figsize = (16, 5), xlabel = 'Temperature', ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
    plt = FvT_gen.result.plot(x = 0, y = UnitNo, kind = 'line', figsize = (16, 5), xlabel = 'Temperature', ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
    plt.plot(fvt_limX, fvt_limY_pos, '--r', label = 'FvT Limit+')
    plt.plot(fvt_limX, fvt_limY_neg, '--r', label = 'FvT Limit-')
    #plt.xaxis.set_ticks(np.arange(-45, 105, 20))
    #plt.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
    plt.legend()
    save_button.on_click(save_on_click)
    ok_button.on_click(ok_on_click)
    temp_data = locals()['UnitNo'] ## store real-time local 'UnitNo' argument value to global 'temp' using locals()
    #save_button.on_click(save_on_click(UnitNo))
    #ui = widgets.HBox([save_button, out])
#save_button.on_click(save_on_click)


# cols_2[0] = 1
# n = 1
# while(n < len(cols_2)):
#     cols_2[n] = cols_2[n-1] + 1
#     n = n + 1
# tuple_Max = max(cols_2)
# buttons = widgets.HBox([save_button, ok_button])
# display(widgets.HBox([buttons, out]))
# interactive_plot = interactive(multiplot, UnitNo= (1, tuple_Max, 1))
# output = interactive_plot.children[-1]
# #output.layout.height = '350px'
# interactive_plot
    
    
    
def clicked_start(event):
    with output:
        clear_output()
        temp.device_des = labl_1.value
        temp.freq = labl_2.value
        temp.sup_vol = labl_3.value
        temp.output_type = labl_4.value
        temp.dev_code = labl_5.value
        temp.rev_day = labl_6.value
        temp.rev_month = labl_61.value
        temp.rev_year = labl_611.value
        temp.n = labl_7.value
        temp.so_num = labl_8.value
        temp.name = labl_9.value
        temp.ebt_num = labl_10.value
        temp.test_cond = labl_11.value
        temp.test_set_1 = labl_12.value
        temp.fvt_spec = float(labl_13.value)
        temp.temp_range = labl_14.value
        temp.test1_cond = labl_15.value
        temp.test1_set_1 = labl_16.value
        temp.plot_num = labl_17.value
        temp.pn_spec = labl_18.value
        temp.test2_cond = labl_19.value
        temp.test2_set_1 = labl_20.value
        temp.save_path = labl_21.value+'.docx'
        temp.pn_fl_type = labl_22.value
        global template
        template = DocxTemplate(rept_template.selected)
        
    
        try:
            temp.temp_range = temp.temp_range.split(' ')
            temp.temp_range = list(map(int, temp.temp_range))
        
        except:
            print("Wrong Operating temperature range!")
    
    
        try:
            temp.pn_spec = temp.pn_spec.split(' ')
            temp.pn_spec = list(map(int, temp.pn_spec))
        
        except:
            print("Wrong PN limit!")
            
        global context_con, context   
        context_con = {
            'test_cond_1': temp.test_cond,
            'test1_cond_1': temp.test1_cond,
            'test2_cond_1': temp.test2_cond
            }

        #print(context_con)    

        context = {
            'device_des': temp.device_des,
            'freq': temp.freq,
            'sup_vol': temp.sup_vol,
            'output_type': temp.output_type,
            'Code': temp.dev_code,
            'day': temp.rev_day,
            'month': temp.rev_month,
            'year': temp.rev_year,
            'n': temp.n,
            'so_num': temp.so_num,
            'd': datetime.datetime.now().strftime('%d'),
            'm': datetime.datetime.now().strftime('%b'),
            'y': datetime.datetime.now().strftime('%Y'),
            'Author': temp.name,
            'test_set_1': temp.test_set_1,
            'test1_set_1': temp.test1_set_1,
            'test2_set_1': temp.test2_set_1
            }

        FvT_gen()

        global fvt_limX, fvt_limY_pos, fvt_limY_neg, save_button, ok_button, out
        save_button = widgets.Button(description = 'Select this Unit data')
        ok_button = widgets.Button(description='Proceed')
        out = widgets.Output()

        temp_data = 0
        fvt_limX = [-48, 108]
        fvt_limY_pos = [temp.fvt_spec, temp.fvt_spec]
        fvt_limY_neg = [-temp.fvt_spec, -temp.fvt_spec]
        saved_cnt = 0
        #ok_flag = 0
        cols_2[0] = 1
        n = 1
        while(n < len(cols_2)):
            cols_2[n] = cols_2[n-1] + 1
            n = n + 1
        tuple_Max = max(cols_2)
        buttons = widgets.HBox([save_button, ok_button])
        display(widgets.HBox([buttons, out]))
        #interactive_plot = interactive(multiplot, UnitNo)
        interactive_plot = interact(multiplot, UnitNo= (1, tuple_Max, 1) )
        #output_gen = interactive_plot.children[-1]
        #output.layout.height = '350px'
        interactive_plot
                
            #sys.exit()
    
labl_1 = widgets.Text(description='Code/ FG Device Description')
labl_2 = widgets.Text(description='Frequency')
labl_3 = widgets.Text(description='Supply Voltage', placeholder='V')
labl_4 = widgets.Text(description='Output Type')
labl_5 = widgets.Text(description='Device Code')
labl_6 = widgets.Text(description='Spec Issue Rev date', placeholder='Day')
labl_61 = widgets.Text(placeholder='Month')
labl_611 = widgets.Text(placeholder='Year')
labl_7 = widgets.Text(description='Sample Quantity')
labl_8 = widgets.Text(description='SO#')
labl_9 = widgets.Text(description='Author')
labl_10 = widgets.IntText(description='EBT tested Quantity')
labl_11 = widgets.Textarea(description='Test Conditions')
labl_12 = widgets.Textarea(description='Test Setup')
labl_13 = widgets.IntText(description='FvT SPEC Limit(ppm)')
labl_14 = widgets.Text(description='Operating Temperature Range')
labl_15 = widgets.Textarea(description='Test Conditions')
labl_16 = widgets.Textarea(description='Test Setup')
labl_17 = widgets.IntText(description='No. of P.N plots')
labl_18 = widgets.Text(description="Typ. at '1Hz 10Hz 100Hz 1kHz 10kHz 100kHz 1MHz'")
labl_19 = widgets.Textarea(description='Test Conditions')
labl_20 = widgets.Textarea(description='Test Setup')
labl_21 = widgets.Text(description='Name your Sample Data Report')
labl_22 = widgets.ToggleButtons(options=['csv', 'txt'])
labl_23 = widgets.FileUpload(accept='', multiple=False)
btn_start = widgets.Button(description='GENERATE REPORT', button_style='success')

output = widgets.Output()
btn_start.on_click(clicked_start)
vbox_result = widgets.VBox([btn_start, output])


#print(temp.pn_spec)
text_4= widgets.HTML(value="<h2>Please select the TEMPLATE report file(.docx). - </h2>")
rept_template = FileChooser()
#print(file_path)
#file_name = ntpath.basename(file_path)
# template = DocxTemplate(rept_template.selected)

text_5= widgets.HTML(value="<h2>Please select the Electical Parameters file(.txt). - </h2>")
starting_directory = 'C:/Xice'
ebt_data = FileChooser(starting_directory)

text_6= widgets.HTML(value="<h2>Please select your raw FvT data(.xlsx). - </h2>")
fvt_data = FileChooser()

text_7= widgets.HTML(value="<h2>Please select your raw Phase Noise file. - </h2>")
pn_data = FileChooser()




text_0 = widgets.HTML(value="<h1>Sample Order Info - </h2>")
text_1 = widgets.HTML(value="<h2>Electrical Performance(EBT) </h2>")
text_2= widgets.HTML(value="<h2>Frequency Stability over Temperature- </h2>")
text_3= widgets.HTML(value="<h2>Phase Noise Performance - </h2>")

vbox_text = widgets.VBox([text_0, labl_1, labl_2, labl_3, labl_4, labl_5, labl_6, labl_61, labl_611, labl_7, labl_8, labl_9,
                          text_1, labl_10, labl_11, labl_12, text_2, labl_13, labl_14, labl_15, labl_16, 
                          text_3, labl_17, labl_18, labl_19, labl_20, labl_21, labl_22, labl_23,
                          text_4, rept_template, text_5, ebt_data, text_6, fvt_data, text_7, pn_data, vbox_result])
page = widgets.HBox([vbox_text])

display(page)


##--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
# def ok_on_click(b):
#     with out:
#         clear_output()
#         #global ok_flag
#         #ok_flag = 1
#         # Rename column labels in unitNo# sequence  
#         unit_qty = len(FvT_gen.res.columns)-1
#         col_lst = ["Operating Temperature(degC)"]
#         for _ in range(unit_qty):
#             col_lst.append("Unit"+ str(_+1))

#         print(col_lst)
#         FvT_gen.res.columns = col_lst
# #---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#         ## finalised plot generated according to saved Units data
#         plt =FvT_gen.res.plot(x = FvT_gen.res.columns[0], y = FvT_gen.res.columns[1:], kind = 'line', figsize = (20, 10), ylabel = 'Norm.Freq(ppm)', title='Sample Frequency Stability', grid = True)
#         #plt =FvT_gen.res.plot(x = 0, y = FvT_gen.res.columns[1:], kind = 'line', figsize = (20, 10), ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
#         plt.plot(fvt_limX, fvt_limY_pos, '--r', label = 'FvT Limit+'+ '(' + str(temp.fvt_spec) + 'ppm)')
#         plt.plot(fvt_limX, fvt_limY_neg, '--r', label = 'FvT Limit-'+ '(' + str(-temp.fvt_spec) + 'ppm)')

#         plt.set_xlim(xmin = temp.temp_range[0], xmax = temp.temp_range[1]) # show preferred x-axis range as per operating temperature
#         plt.set_ylim(ymin = -temp.fvt_spec*2.5, ymax = temp.fvt_spec*2.5) # Keep the plot centered within scope

#         #plt.xaxis.set_ticks(np.arange(-45, 105, 20))
#         #plt.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
#         plt.legend()
#         fig = plt.get_figure()
#         fig.savefig("FvT_output_0" +'.png')
# ##---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#         context['FvT_Plot_0'] = InlineImage(template, 'FvT_output_0.png', width = Mm(240), height= Mm(120))
#         #context['FvT_Plot_1'] = InlineImage(template, 'FvT_output_1.png', width = Mm(240), height= Mm(120))
# ##---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#         if(temp.pn_fl_type == 'csv'):
#             PN_gen_csv()
#         if(temp.pn_fl_type == 'txt'):
#             PN_gen_txt()
    
#         context['PN_Plot'] = InlineImage(template, 'PN_output.png', width = Mm(240), height= Mm(120))

#         template.render(Merge_context(context, context_con))
#         # save_path = simpledialog.askstring(title="Save", prompt="Please type in the file name with .docx : \t\t\t\t\t\t")
#         template.save(temp.save_path)
#         get_Table_1(temp.save_path)
        
# def save_on_click(b):
#     with out:
#         #FvT_gen.df2.plot(x = 'Temp(deg)', y = UnitNo, kind = 'line', figsize = (16, 5)).savefig(f'FvT.png')
#         #print(FvT_gen.df2[b])
#         #print(111)
#         clear_output()
#         #print(FvT_gen.result[temp_data])
#         print("\n.............................................................................................\n")
#         print("\nYOUR SAVED UNIT DATA:\n")
#         FvT_gen.res[temp_data] = FvT_gen.result[temp_data] ## update selected Unit data into the df3
#         print(FvT_gen.res)
#         global saved_cnt
#         saved_cnt = len(FvT_gen.res.columns)-1
#         print("Saved Counter - ", saved_cnt)
        
# def multiplot(UnitNo):
#     global temp_data ## the global 'temp' variable 
#     opts = FvT_gen.result.columns.values
#     #plt = FvT_gen.result.plot(x = 0, y = 1, kind = 'line', figsize = (16, 5), xlabel = 'Temperature', ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
#     plt = FvT_gen.result.plot(x = 0, y = UnitNo, kind = 'line', figsize = (16, 5), xlabel = 'Temperature', ylabel = 'Norm.Freq', title='Sample Frequency Stability', grid = True)
#     plt.plot(fvt_limX, fvt_limY_pos, '--r', label = 'FvT Limit+')
#     plt.plot(fvt_limX, fvt_limY_neg, '--r', label = 'FvT Limit-')
#     #plt.xaxis.set_ticks(np.arange(-45, 105, 20))
#     #plt.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
#     plt.legend()
#     save_button.on_click(save_on_click)
#     ok_button.on_click(ok_on_click)
#     temp_data = locals()['UnitNo'] ## store real-time local 'UnitNo' argument value to global 'temp' using locals()
#     #save_button.on_click(save_on_click(UnitNo))
#     #ui = widgets.HBox([save_button, out])
# #save_button.on_click(save_on_click)
    
# cols_2[0] = 1
# n = 1
# while(n < len(cols_2)):
#     cols_2[n] = cols_2[n-1] + 1
#     n = n + 1
# tuple_Max = max(cols_2)
# buttons = widgets.HBox([save_button, ok_button])
# display(widgets.HBox([buttons, out]))
# interactive_plot = interactive(multiplot, UnitNo= (1, tuple_Max, 1))
# output = interactive_plot.children[-1]
# #output.layout.height = '350px'
# interactive_plot